To copy this template: File -> Save a Copy in Drive

***DISCLAIMER**: In case of any discrepancy in the assignment instruction, please refer to the `PDF` document.*

# Problem 1 - Sentiment Analysis using recurrent models

## 5.1

In [ ]:
import torchvision
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import tensorflow
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, Embedding
from tensorflow.keras.optimizers import Adam


2023-11-13 20:13:25.951066: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-13 20:13:28.542038: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-13 20:13:28.542072: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-13 20:13:28.542113: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-13 20:13:28.624291: I tensorflow/core/platform/cpu_feature_g

In [ ]:
df = pd.read_csv("IMDB Dataset.csv", usecols=["review", "sentiment"], encoding='latin-1')
## 1 - positive, 0 - negative
df.sentiment = (df.sentiment == "positive").astype("int")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
val_size = int(df.shape[0] * 0.15)
test_size = int(df.shape[0] * 0.15)


def train_val_test_split(df=None, train_percent=0.7, test_percent=0.15, val_percent=0.15):
  df = df.sample(frac=1)
  train_df = df[: int(len(df)*train_percent)]
  test_df = df[int(len(df)*train_percent)+1 : int(len(df)*(train_percent+test_percent))]
  val_df = df[int(len(df)*(train_percent + test_percent))+1 : ]
  return train_df, test_df, val_df

train_df, test_df, val_df = train_val_test_split(df, 0.7, 0.15, 0.15)
train_labels, train_texts = train_df.values[:,1], train_df.values[:,0]
val_labels, val_texts = val_df.values[:,1], val_df.values[:,0]
test_labels, test_texts = test_df.values[:,1], test_df.values[:,0]
print(len(train_df), len(test_df), len(val_df))
print(len(train_texts), len(train_labels), len(val_df))

35000 7499 7499
35000 35000 7499


In [ ]:
train_texts[:5]

array(['You have to be awfully patient to sit through a film with one-liners so flat and unfunny that you wonder what all the fuss was about when WHISTLING IN THE DARK opened to such an enthusiastic greeting from audiences in the 1940s.<br /><br />On top of some weak one-liners and ordinary sight gags, the plot is as far-fetched as the tales The Fox (Red Skelton) tells his radio audience. You have to wonder why anyone would think he could come up with a real-life solution on how to commit the perfect crime and get away with it. But then, that\'s how unrealistic the comedy is.<br /><br />But--if you\'re a true Red Skelton fan and enjoy a look back at how comedies were made in the \'40s--you can at least enjoy the amiable cast supporting him. Ann Rutherford and Virginia Grey do nicely as his love interest and Conrad Veidt, as always, makes an interesting villain. One of his more amusing moments is his reaction to Skelton explaining the mysteries of wearing turbans. "I never knew that," h

In [ ]:

from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

def process_tokens(text):
    """
    function to process tokens, replace any unwanted chars
    """
    preprocessed_text = text.lower().replace(",", "").replace(".", "").replace(":", "").replace(")", "").replace("-", "").replace("(", "")
    preprocessed_text = ''.join([i for i in preprocessed_text if not preprocessed_text.isdigit()])
    return preprocessed_text

def preprocessing(data):
    """
    preprocessing data to list of tokens
    """
    nlp = English()
    tokenizer = Tokenizer(nlp.vocab)
    preprocessed_data = []
    for sentence in data:
        sentence = process_tokens(sentence)
        tokens = tokenizer(sentence)
        tlist = []
        for token in tokens:
            tlist.append(str(token))
        preprocessed_data.append(tlist)
    return preprocessed_data

train_data = preprocessing(train_texts)
val_data = preprocessing(val_texts)
test_data = preprocessing(test_texts)

In [ ]:
print(train_data[0])

['you', 'have', 'to', 'be', 'awfully', 'patient', 'to', 'sit', 'through', 'a', 'film', 'with', 'oneliners', 'so', 'flat', 'and', 'unfunny', 'that', 'you', 'wonder', 'what', 'all', 'the', 'fuss', 'was', 'about', 'when', 'whistling', 'in', 'the', 'dark', 'opened', 'to', 'such', 'an', 'enthusiastic', 'greeting', 'from', 'audiences', 'in', 'the', '1940s<br', '/><br', '/>on', 'top', 'of', 'some', 'weak', 'oneliners', 'and', 'ordinary', 'sight', 'gags', 'the', 'plot', 'is', 'as', 'farfetched', 'as', 'the', 'tales', 'the', 'fox', 'red', 'skelton', 'tells', 'his', 'radio', 'audience', 'you', 'have', 'to', 'wonder', 'why', 'anyone', 'would', 'think', 'he', 'could', 'come', 'up', 'with', 'a', 'reallife', 'solution', 'on', 'how', 'to', 'commit', 'the', 'perfect', 'crime', 'and', 'get', 'away', 'with', 'it', 'but', 'then', "that's", 'how', 'unrealistic', 'the', 'comedy', 'is<br', '/><br', '/>butif', "you're", 'a', 'true', 'red', 'skelton', 'fan', 'and', 'enjoy', 'a', 'look', 'back', 'at', 'how', '

In [ ]:
import numpy as np
import itertools

## Creating a vectorizer to vectorize text and create matrix of features
## Bag of words technique
class Vectorizer():
    def __init__(self, max_features):
        self.max_features = max_features
        self.vocab_list = None
        self.token_to_index = None

    def fit(self, dataset):
        word_dict = {}
        for sentence in dataset:
            for token in sentence:
                if token not in word_dict:
                    word_dict[token] = 1
                else:
                    word_dict[token] += 1
        word_dict = dict(sorted(word_dict.items(), key=lambda item: item[1], reverse=True))
        end_to_slice = min(len(word_dict), self.max_features)
        word_dict = dict(itertools.islice(word_dict.items(), end_to_slice))
        self.vocab_list = list(word_dict.keys())
        self.token_to_index = {}
        counter = 0
        for token in self.vocab_list:
            self.token_to_index[token] = counter
            counter += 1


    def transform(self, dataset):
        data_matrix = np.zeros((len(dataset), len(self.vocab_list)))
        for i, sentence in enumerate(dataset):
            for token in sentence:
                if token in self.token_to_index:
                    data_matrix[i, self.token_to_index[token]] += 1
        return data_matrix

## max features - top k words to consider only
max_features = 2000

vectorizer = Vectorizer(max_features=max_features)
vectorizer.fit(train_data)

## Checking if the len of vocab = k
X_train = vectorizer.transform(train_data)
X_val = vectorizer.transform(val_data)
X_test = vectorizer.transform(test_data)

y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)

vocab = vectorizer.vocab_list

In [ ]:
## each sequence of token is a vector of
## token indices (with the count of those words)
X_train[:5]

array([[16.,  8.,  6., ...,  0.,  0.,  0.],
       [25., 12.,  9., ...,  0.,  0.,  0.],
       [18.,  7.,  6., ...,  0.,  0.,  0.],
       [28., 13., 20., ...,  0.,  0.,  0.],
       [ 1.,  1.,  3., ...,  0.,  0.,  0.]])

In [ ]:
y_train = y_train.astype('int')
y_val = y_val.astype('int')
y_test = y_test.astype('int')

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)
y_val = to_categorical(y_val, 2)

In [ ]:
X_train = X_train.reshape(-1, 1, X_train.shape[1])
X_val = X_val.reshape(-1, 1, X_val.shape[1])
X_test = X_test.reshape(-1, 1, X_test.shape[1])

y_train = y_train.reshape(-1, 2)
y_val = y_val.reshape(-1, 2)
y_test = y_test.reshape(-1, 2)

print(f'X_train.shape: {X_train.shape}, y_train.shape: {y_train.shape}')

X_train.shape: (35000, 1, 2000), y_train.shape: (35000, 2)


## 5.2

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN, Dropout
from tensorflow.keras.optimizers import Adam

model = None
model = Sequential()
model.add(SimpleRNN(256, input_shape=(1, max_features)))
model.add(Dense(2, activation='softmax'))

optimizer = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,
              metrics=['accuracy'])
print(model.summary())
history = model.fit(X_train, y_train,
          batch_size=256,
          validation_data=(X_val, y_val),
          epochs=10)
print(history.history.keys())

2023-11-13 20:14:44.216329: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 44333 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:06:00.0, compute capability: 7.5


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 256)               577792    
                                                                 
 dense (Dense)               (None, 2)                 514       
                                                                 
Total params: 578306 (2.21 MB)
Trainable params: 578306 (2.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10


2023-11-13 20:14:48.871673: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x151d34909b90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-13 20:14:48.871707: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 8000, Compute Capability 7.5
2023-11-13 20:14:48.877663: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-13 20:14:49.223943: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2023-11-13 20:14:49.333203: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


137/137 [==============================] - 4s 9ms/step - loss: 0.5389 - accuracy: 0.8077 - val_loss: 0.3048 - val_accuracy: 0.8672
Epoch 2/10
137/137 [==============================] - 1s 7ms/step - loss: 0.2667 - accuracy: 0.8885 - val_loss: 0.2911 - val_accuracy: 0.8733
Epoch 3/10
137/137 [==============================] - 1s 7ms/step - loss: 0.2276 - accuracy: 0.9064 - val_loss: 0.2996 - val_accuracy: 0.8729
Epoch 4/10
137/137 [==============================] - 1s 6ms/step - loss: 0.1798 - accuracy: 0.9287 - val_loss: 0.4054 - val_accuracy: 0.8476
Epoch 5/10
137/137 [==============================] - 1s 6ms/step - loss: 0.1237 - accuracy: 0.9553 - val_loss: 0.3688 - val_accuracy: 0.8684
Epoch 6/10
137/137 [==============================] - 1s 7ms/step - loss: 0.0661 - accuracy: 0.9809 - val_loss: 0.4072 - val_accuracy: 0.8708
Epoch 7/10
137/137 [==============================] - 1s 7ms/step - loss: 0.0380 - accuracy: 0.9908 - val_loss: 0.4466 - val_accuracy: 0.8717
Epoch 8/10
137/13

In [ ]:
score, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score)
print('Test accuracy:', acc)

Test loss: 0.555500328540802
Test accuracy: 0.8751833438873291


## 5.3

In [ ]:
from tensorflow.keras.layers import LSTM

model = None
model = Sequential()
model.add(LSTM(256, input_shape=(1, max_features)))
model.add(Dense(2, activation='softmax'))

optimizer = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,
              metrics=['accuracy'])
print(model.summary())
history = model.fit(X_train, y_train,
          batch_size=256,
          validation_data=(X_val, y_val),
          epochs=10)
print(history.history.keys())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               2311168   
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 2311682 (8.82 MB)
Trainable params: 2311682 (8.82 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10
137/137 [==============================] - 5s 13ms/step - loss: 0.3731 - accuracy: 0.8330 - val_loss: 0.3102 - val_accuracy: 0.8677
Epoch 2/10
137/137 [==============================] - 1s 7ms/step - loss: 0.2673 - accuracy: 0.8871 - val_loss: 0.2999 - val_accuracy: 0.8704
Epoch 3/10
137/137 [==============================] - 1s 6ms/step - loss: 0.2217 - accuracy: 0.9076 - val_loss: 0.3063 - v

In [ ]:
score, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score)
print('Test accuracy:', acc)

Test loss: 0.5813266634941101
Test accuracy: 0.8675823211669922


## 5.4

In [ ]:
# getting data ready
X_train = vectorizer.transform(train_data)
X_val = vectorizer.transform(val_data)
X_test = vectorizer.transform(test_data)

y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)

In [ ]:
y_train = y_train.astype('int')
y_val = y_val.astype('int')
y_test = y_test.astype('int')

In [ ]:
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)
y_val = to_categorical(y_val, 2)

In [ ]:
X_train = X_train.reshape(-1, 1, X_train.shape[1])
X_val = X_val.reshape(-1, 1, X_val.shape[1])
X_test = X_test.reshape(-1, 1, X_test.shape[1])

y_train = y_train.reshape(-1, 2)
y_val = y_val.reshape(-1, 2)
y_test = y_test.reshape(-1, 2)

print(f'X_train.shape: {X_train.shape}, y_train.shape: {y_train.shape}')

X_train.shape: (35000, 1, 2000), y_train.shape: (35000, 2)


In [ ]:
from tensorflow.keras.layers import GRU

model = None
model = Sequential()
model.add(GRU(256, input_shape=(1, max_features)))
model.add(Dense(2, activation='softmax'))

optimizer = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,
              metrics=['accuracy'])
print(model.summary())
history = model.fit(X_train, y_train,
          batch_size=256,
          validation_data=(X_val, y_val),
          epochs=10)
print(history.history.keys())

score, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score)
print('Test accuracy:', acc)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 256)               1734144   
                                                                 
 dense_2 (Dense)             (None, 2)                 514       
                                                                 
Total params: 1734658 (6.62 MB)
Trainable params: 1734658 (6.62 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10
137/137 [==============================] - 3s 9ms/step - loss: 0.3707 - accuracy: 0.8465 - val_loss: 0.3013 - val_accuracy: 0.8706
Epoch 2/10
137/137 [==============================] - 1s 7ms/step - loss: 0.2574 - accuracy: 0.8916 - val_loss: 0.2939 - val_accuracy: 0.8789
Epoch 3/10
137/137 [==============================] - 1s 6ms/step - loss: 0.2080 - accuracy: 0.9147 - val_loss: 0.3144 - va

In [ ]:
# check predictions
from tensorflow.keras.backend import argmax

y_pred = model.predict(X_test)
for i in range(5):
  print(f'Label predicted: {argmax(y_pred[i]).numpy()}, Actual label: {argmax(y_test[i]).numpy()}')
  print(f'text: {test_texts[i]}')

235/235 [==============================] - 1s 2ms/step
Label predicted: 1, Actual label: 1
text: Although this film is somewhat sanitized (because it was made at a time when people just didn't talk about sex), it is an extremely helpful short film to show prepubescent girls so they know what to expect during menstruation. Not surprisingly, it was paid for by the Kotex company, though what may surprise many is that Disney made this film--as they made a lot of educational films during the 1940s-60s. However well made the film is, though, I think the film maker's missed a real opportunity. Instead of the nice female narrator's voice and the relatively bland visuals it would have been GREAT if they'd used Minnie Mouse and the rest of the Disney gang!! I know this would have given old Walt a heart attack, but wow that would have been a great film! By the way, although the notion of sex is barely hinted at in the film, it DOES adequately explain menstruation in general. However, it does lack

## 5.5

In [ ]:
from tensorflow.keras.layers import LSTM

model = None
model = Sequential()
model.add(Bidirectional(LSTM(256), input_shape=(1, max_features)))
model.add(Dense(2, activation='softmax'))

optimizer = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,
              metrics=['accuracy'])
print(model.summary())
history = model.fit(X_train, y_train,
          batch_size=256,
          validation_data=(X_val, y_val),
          epochs=10)
print(history.history.keys())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirecti  (None, 512)               4622336   
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 4623362 (17.64 MB)
Trainable params: 4623362 (17.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10
137/137 [==============================] - 4s 13ms/step - loss: 0.4118 - accuracy: 0.8213 - val_loss: 0.3001 - val_accuracy: 0.8688
Epoch 2/10
137/137 [==============================] - 1s 8ms/step - loss: 0.2597 - accuracy: 0.8931 - val_loss: 0.2920 - val_accuracy: 0.8779
Epoch 3/10
137/137 [==============================] - 

In [ ]:
score, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score)
print('Test accuracy:', acc)

Test loss: 0.5876869559288025
Test accuracy: 0.8723829984664917


## 5.6

**Answer:**

RNN
 - Test Loss: 0.555500328540802
 -Test Accuracy: 0.8751833438873291

LSTM
   - Test Loss: 0.5813266634941101
   - Test Accuracy: 0.8675823211669922

GRU
   - Test Loss: 0.6560810208320618
   - Test Accuracy: 0.8710494637489319

BiLSTM
   - Test Loss: 0.5876869559288025
   - Test Accuracy: 0.8723829984664917


The RNN model has the lowest test loss and highest accuracy of the four models at  0.555500 and 0.8751833 respectively. This is a bit unusual.
The RNN's performance indicates that this dataset with these conditions the more simple model accurately the important information for sentiment analysis and more complex models did provide an advantage.

Here the LSTM model had about 0.03 more test loss and about 0.1 less test accuracy. The GRU model had 0.1 more test loss and about 0.004 less accuracy. The BiLSTM model had about 0.03 more test loss and a 0.003 less accuracy.

LSTM and GRU models are meant to prevent the vanishing gradient probelem and the RNN and BiLSTM models are meant to find the patterns that go both directions of the sequence. Potentially the learning rate may not have been opitmal for the three other models.
